In [4]:
from postgres_connect import *
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
# from weasyprint import HTML
from fpdf import FPDF
# from tc_pdf import TCPDF
import pdfkit
import pandas as pd

In [5]:
path = '/Users/apple/Documents/tunnel-ssh.cer'
# printthis('yes')
conn = get_conn('Yes',path)

Tunnel Started
Connection Made


In [7]:
query = '''
select 
    shipping_city
,   hub
,   rider_id
,   rider_name
,   need_improvement
,   starting_fasr
from public.rider_pip
where pip_date = '2023-12-11'
'''

In [8]:
class PDF(FPDF):
    def header(self):
        pass  # No header needed

    def footer(self):
        pass  # No footer needed

    def chapter_title(self, rider_name):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, f'Dear {rider_name},', 0, 1, 'L')
        self.ln(4)

    def chapter_body(self, content):
        self.set_font('Arial', '', 12)
        self.multi_cell(0, 10, content)
        self.ln()

def create_pdf(rider_name, need_improvement, output_path, pdf):

    pdf.set_font('Arial', 'BU', 14)
    pdf.cell(200, 10, txt=f"Dear {rider_name},", ln=True)
    pdf.set_font('Arial','', 12)
    pdf.multi_cell(200, 10, "Your dedication is valued. Based on your performance last week, "
                             "you have been placed in a Performance Improvement Program.")
    pdf.multi_cell(200, 10, "This is an opportunity for improvement, but it's also a critical time. "
                             "Your success is crucial to us. By the end of this week, your performance will be monitored, "
                             "and based on the outcome, there's a possibility of termination from the company.")
    pdf.multi_cell(200, 10, "We regret the need for such action, but support and assistance are available throughout.")


    # Targets
    pdf.set_font('Arial', 'B', 12)
    pdf.multi_cell(0, 10, f"You need Improvement here: {need_improvement}")
    pdf.multi_cell(0, 10, f"Target -\nMorning: Prepaid FASR 85%, COD FASR 60%\nEvening: Prepaid FASR 80%, COD FASR 50%")
    pdf.ln()

    # Your Name
    pdf.set_font('Arial', '', 12)
    pdf.multi_cell(0, 10, "All the Best!")



def generate_pip_pdfs_by_hub(df_pip):
    hubs = df_pip['hub'].unique()

    for hub in hubs:
        hub_df = df_pip[df_pip['hub'] == hub]
        output_path = f"{hub}_Performance_Improvement_Program.pdf"
        pdf = PDF()
        pdf.add_page()
        with pd.option_context('mode.chained_assignment', None):
            hub_df.reset_index(drop=True, inplace=True)

        for index, row in hub_df.iterrows():
            rider_name = row['rider_name']
            need_improvement = row['need_improvement']
            create_pdf(rider_name, need_improvement, output_path, pdf)
            pdf.add_page()
        pdf.output(output_path)

def main():
    df_pip = pd.read_sql(query,conn)
    print(df_pip)
    generate_pip_pdfs_by_hub(df_pip)
    conn.close

if __name__ == "__main__":
    main()


   shipping_city          hub                      rider_id  \
0      Bangalore  DS BLR BOMM                      Akash__2   
1      Bangalore  DS BLR BOMM               praveen_Reddy_2   
2      Bangalore  DS BLR BOMM                Shashank_Rao_2   
3      Bangalore  DS BLR HBBL                  Gangadhara_7   
4      Bangalore  DS BLR HBBL         Syed_Muzammil_pasha_7   
5      Bangalore  DS BLR HBBL                     Harisha_7   
6      Bangalore  DS BLR HBBL         Sachain_Anand_Lodhi_7   
7      Bangalore   DS BLR MTH  BOMM_SmarakKumarDAS_03102022   
8      Bangalore   DS BLR MTH   DILIP_KUMAR_TATA_ACE_MRTH_1   
9          Delhi   DS DEL DWK                     Rahul_2_5   
10         Delhi   DS DEL DWK        Akhlekh_Ansari_Laksh_5   
11         Delhi   DS DEL DWK        subham_saxena_van_de_5   
12         Delhi   DS DEL DWK                 Rahul_Laksh_5   
13         Delhi   DS DEL DWK                   sadeep_ps_5   
14         Delhi   DS DEL DWK           Sudhir_Rajak_LV

In [24]:
class PDF(TCPDF):
    def header(self):
        pass  # No header needed

    def footer(self):
        pass  # No footer needed

    def chapter_title(self, rider_name):
        self.set_font('AdelleSansDevanagari-Regular', '', 12)
        self.cell(0, 10, f'प्रिय {rider_name},', 0, 1, 'L')
        self.ln(4)

    def chapter_body(self, content):
        self.set_font('AdelleSansDevanagari-Regular', '', 12)
        self.multi_cell(0, 10, content)
        self.ln()

def create_pdf(rider_name, need_improvement, output_path, pdf, is_english=True):
    if is_english:
        pdf.set_font('Arial', 'BU', 14)
        pdf.cell(200, 10, txt=f"Dear {rider_name},", ln=True)
        pdf.set_font('Arial', '', 12)
        pdf.multi_cell(200, 10, "Your dedication is valued. Based on your performance last week, "
                                 "you have been placed in a Performance Improvement Program.")
        pdf.multi_cell(200, 10, "This is an opportunity for improvement, but it's also a critical time. "
                                 "Your success is crucial to us. By the end of this week, your performance will be monitored, "
                                 "and based on the outcome, there's a possibility of termination from the company.")
        pdf.multi_cell(200, 10, "We regret the need for such action, but support and assistance are available throughout.")
    else:
        pdf.set_font('AdelleSansDevanagari-Regular', 'BU', 14)
        pdf.cell(200, 10, txt=f"प्रिय {rider_name},", ln=True)
        pdf.set_font('AdelleSansDevanagari-Regular', '', 12)
        pdf.multi_cell(200, 10, "मुझे आशा है कि आप स्वस्थ हैं। आपका समर्पण मूल्यवान है। पिछले हफ्ते के प्रदर्शन के आधार पर, आपको एक प्रदर्शन सुधार कार्यक्रम में रखा गया है।")
        pdf.multi_cell(200, 10, "यह सुधार का एक अवसर है, लेकिन यह भी एक महत्वपूर्ण समय है। आपकी सफलता हमारे लिए अत्यंत महत्वपूर्ण है। इस हफ्ते के अंत तक, आपका प्रदर्शन मॉनिटर किया जाएगा, और इसके परिणामस्वरूप, कंपनी से समाप्ति की संभावना है।")
        pdf.multi_cell(200, 10, "हम इस कदम की आवश्यकता के लिए खेद करते हैं, लेकिन समर्थन और सहायता पूरे के पूरे उपलब्ध हैं।")

    # Targets
    pdf.set_font('Arial', 'B', 12)
    pdf.multi_cell(0, 10, f"You need Improvement here: {need_improvement}")
    pdf.multi_cell(0, 10, f"Target -\nMorning: Prepaid FASR 85%, COD FASR 60%\nEvening: Prepaid FASR 80%, COD FASR 50%")
    pdf.ln()

    # Your Name
    pdf.set_font('Arial', '', 12)
    pdf.multi_cell(0, 10, "All the Best!")

def generate_pip_pdfs_by_hub(df_pip):
    hubs = df_pip['hub'].unique()

    for hub in hubs:
        hub_df = df_pip[df_pip['hub'] == hub]
        output_path = f"{hub}_Performance_Improvement_Program.pdf"
        pdf = PDF()
        pdf.add_page()
        
        with pd.option_context('mode.chained_assignment', None):
            hub_df.reset_index(drop=True, inplace=True)

        for index, row in hub_df.iterrows():
            rider_name = row['rider_name']
            need_improvement = row['need_improvement']
            
            # Hindi PDF
            create_pdf(rider_name, need_improvement, output_path, pdf, is_english=False)
            pdf.add_page()
            
            # English PDF
            create_pdf(rider_name, need_improvement, output_path, pdf, is_english=True)
            pdf.add_page()
            

        pdf.output(output_path)

def main():
    df_pip = pd.read_sql(query, conn)
    print(df_pip)
    generate_pip_pdfs_by_hub(df_pip)
    conn.close()

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'tcpdf'